In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.utils import to_categorical
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import ConfusionMatrixDisplay

from keras.optimizers import SGD

In [ ]:
# Load the data
data_1 = pd.read_csv('EEG_mouse_data_1.csv')
data_2 = pd.read_csv('EEG_mouse_data_2.csv')
data = pd.concat([data_1, data_2])

# Preprocess the data
features = data.columns[1:26]  # Select 25 features
X = data[features].values
y = data['state'].values  # Target variable

# First experiment: awake / asleep

In [29]:
# Group 'n-rem' and 'rem' into 'asleep'
y = data['state'].replace(['n', 'r'], 'asleep').values


# Convert labels to integer values
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Convert integer labels to categorical
y = to_categorical(y)

# Define the MLP model
model = Sequential()
model.add(Input(shape=(25,))) # Input layer with 25 features
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(y.shape[1], activation='sigmoid'))  # Number of neurons equals number of classes

# Define your learning rate
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.1)

# Pass the learning rate to the optimizer
sgd = SGD(learning_rate=lr_schedule, momentum=0.9)

model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

# Use 3-fold cross-validation
kf = KFold(n_splits=3, shuffle=True)
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Normalize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)  # Fit and transform the training data
    X_val = scaler.transform(X_val)  # Transform the validation data

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32)

    # Plot training and validation loss
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Losses')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Evaluate the model
    y_pred = model.predict(X_val)
    
    print('Confusion Matrix:\n')
    # Compute confusion matrix
    cm = confusion_matrix(y_val.argmax(axis=1), y_pred.argmax(axis=1))

    # Create ConfusionMatrixDisplay instance
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=encoder.classes_)

    # Plot confusion matrix with black text
    disp.plot(cmap='binary', colorbar=False, values_format='d')
    plt.show()
        
    confusion_matrix(y_val.argmax(axis=1), y_pred.argmax(axis=1))
    
    f1_scores_per_class = f1_score(y_val.argmax(axis=1), y_pred.argmax(axis=1), average=None)
    f1_score_micro = f1_score(y_val.argmax(axis=1), y_pred.argmax(axis=1), average='micro')

    print('F1 Score per class: ', f1_scores_per_class)
    print('Micro F1 Score: ', f1_score_micro)
    
# Save the model's predictions on the test set
test_data = pd.read_csv('EEG_mouse_data_test.csv')
X_test = test_data[features].values  # Use the same features as the training data
X_test = scaler.transform(X_test)  # Use the same scaler fitted on the last training fold
test_predictions = model.predict(X_test)

model.summary()


with open('test_pred.npy', 'wb') as f:
    np.save(f, test_predictions.argmax(axis=1))

Epoch 1/30
852/852 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6046 - loss: 0.2373 - val_accuracy: 0.6846 - val_loss: 0.2068
Epoch 2/30
852/852 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6995 - loss: 0.2034 - val_accuracy: 0.7418 - val_loss: 0.1970
Epoch 3/30
852/852 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7361 - loss: 0.1976 - val_accuracy: 0.7487 - val_loss: 0.1954
Epoch 4/30
 83/852 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7504 - loss: 0.1948

KeyboardInterrupt: 

# Second experiment: awake / n-rem / rem

# Third experiment: competition 